In [1]:
import os, warnings, logging, itertools, json

import tensorflow as tf
import pandas as pd
import numpy as np

from data_loader import data_loader
from evaluator import compute_error, stepwise_error
from tqdm.notebook import tqdm
from MG_TAR import MG_TAR

logging.disable(logging.WARNING)
warnings.filterwarnings('ignore')

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [3]:
city = 'Seoul'
year = '2016' # '2016' or '2018'
n_steps, length = 6, 12

In [4]:
metric = 'jaccard'

with open('./model_configs.json') as f:
    model_configs = json.load(f)

In [5]:
datasets = data_loader('./datasets', city, year, length=length, n_steps=n_steps, is_scale=True, temporal_copy=True)
n_districts = len(datasets['selected_areas'])

# Extract Features to their corresponding variables
risk_train, risk_test = datasets['risk'][0], datasets['risk'][1]
demo_train, demo_test = datasets['demo'][0], datasets['demo'][1]
poi_train, poi_test = datasets['poi'][0], datasets['poi'][1]
road_train, road_test = datasets['road'][0], datasets['road'][1]
volume_train, volume_test = datasets['volume'][0], datasets['volume'][1]
speed_train, speed_test = datasets['speed'][0], datasets['speed'][1]
weather_train, weather_test = datasets['weather'][0], datasets['weather'][1]
calendar_train, calendar_test = datasets['calendar'][0], datasets['calendar'][1]
c_train, c_test = datasets['dtg'][0], datasets['dtg'][1]
y_train, y_test = datasets['y'][0], datasets['y'][1]


# Train - Validation Split
val_idx = round(risk_train.shape[0] * 0.10) # 10% of Train Set
risk_train, risk_val = risk_train[:-val_idx], risk_train[-val_idx:]
demo_train, demo_val = demo_train[:-val_idx], demo_train[-val_idx:]
poi_train, poi_val = poi_train[:-val_idx], poi_train[-val_idx:]
road_train, road_val = road_train[:-val_idx], road_train[-val_idx:]
volume_train, volume_val = volume_train[:-val_idx], volume_train[-val_idx:]
speed_train, speed_val = speed_train[:-val_idx], speed_train[-val_idx:]
weather_train, weather_val = weather_train[:-val_idx], weather_train[-val_idx:]
calendar_train, calendar_val = calendar_train[:-val_idx], calendar_train[-val_idx:]
c_train, c_val = c_train[:-val_idx], c_train[-val_idx:]
y_train, y_val = y_train[:-val_idx], y_train[-val_idx:]

# Contextual & Adjacency Matrices
A = pd.read_csv(f'./datasets/graph_data/{city}-normalized-district.csv', engine='c', index_col=0).to_numpy()
A_poi = pd.read_csv(f'./datasets/graph_data/{city}-{year}-poi-normalized-district-{metric}.csv', engine='c', index_col=0).to_numpy()
A_demo = pd.read_csv(f'./datasets/graph_data/{city}-{year}-demo-normalized-district-{metric}.csv', engine='c', index_col=0).to_numpy()
A_road = pd.read_csv(f'./datasets/graph_data/{city}-{year}-road-normalized-district-{metric}.csv', engine='c', index_col=0).to_numpy()

A_train, A_val = np.tile(A, (risk_train.shape[0], 1, 1)), np.tile(A, (risk_val.shape[0], 1, 1))
A_poi_train, A_poi_val = np.tile(A_poi, (risk_train.shape[0], 1, 1)), np.tile(A_poi, (risk_val.shape[0], 1, 1))
A_demo_train, A_demo_val = np.tile(A_demo, (risk_train.shape[0], 1, 1)), np.tile(A_demo, (risk_val.shape[0], 1, 1))
A_road_train, A_road_val = np.tile(A_road, (risk_train.shape[0], 1, 1)), np.tile(A_road, (risk_val.shape[0], 1, 1))

A_test = np.tile(A, (risk_test.shape[0], 1, 1))
A_poi_test = np.tile(A_poi, (risk_test.shape[0], 1, 1))
A_demo_test = np.tile(A_demo, (risk_test.shape[0], 1, 1))
A_road_test = np.tile(A_road, (risk_test.shape[0], 1, 1))

with open(f'datasets/graph_data/{city}-{year}-traffic-district-normalized-train-{metric}.npy', 'rb') as f:
    A_traffic_train = np.load(f)
    A_traffic_train, A_traffic_val = A_traffic_train[:-val_idx], A_traffic_train[-val_idx:]

with open(f'datasets/graph_data/{city}-{year}-traffic-district-normalized-test-{metric}.npy', 'rb') as f:
    A_traffic_test = np.load(f)

# Features Aggregation --> inputs=[A, A_poi, A_demo, A_road, A_traffic, X]
node_features_train = np.concatenate([risk_train, demo_train, poi_train, road_train, volume_train, speed_train, weather_train, calendar_train, c_train], axis=-1)
node_features_val = np.concatenate([risk_val, demo_val, poi_val, road_val, volume_val, speed_val, weather_val, calendar_val, c_val], axis=-1)
node_features_test = np.concatenate([risk_test, demo_test, poi_test, road_test, volume_test, speed_test, weather_test, calendar_test, c_test], axis=-1)

x_train = [A_train, A_poi_train, A_demo_train, A_road_train, A_traffic_train, node_features_train]
x_val = [A_val, A_poi_val, A_demo_val, A_road_val, A_traffic_val, node_features_val]
x_test = [A_test, A_poi_test, A_demo_test, A_road_test, A_traffic_test, node_features_test]

# Load Saved Model
best_config = model_configs[year][0]
best_model = MG_TAR(x_train, y_train, x_val, y_val, best_config)

y_pred = best_model.predict(x_test)
sw_errors = stepwise_error(y_test, y_pred, n_steps)

print(f'Model Performance Seoul ({year}):')
print(f"MSE: {np.average(sw_errors['MSE'])} - Acc@K: {np.average(sw_errors['ACC'])} \n")

Model Performance Seoul (2016):
MSE: 2.609439169474812 - Acc@K: 0.39492753623188404 

